In [ ]:
import warnings

warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
warnings.filterwarnings("ignore", ".*dubious year.*")
warnings.filterwarnings(
    "ignore", "Tried to get polar motions for times after IERS data is valid.*"
)

In [ ]:
import logging

import numpy as np
from astropy import units as u
from astropy.table import QTable
from scipy import stats

from earthorbitplan.probability.rate import poisson_lognormal_rate_quantiles

### Setup logging 

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    force=True,
)

In [ ]:
import os

print("cwd =", os.getcwd())
print("events.ecsv exists?", os.path.exists("../data/events.ecsv"))

### Load the Data

In [ ]:
main_table = QTable.read("../data/events.ecsv")

In [ ]:
runs = np.unique(main_table["run"])

print(runs)

### Inspect Plan Arguments and Set Cutoff

In [ ]:
cutoff = main_table["cutoff"]
main_table = main_table[main_table["objective_value"] >= cutoff]
main_table[0:1]

In [ ]:
event_tables_by_run = {run: main_table[main_table["run"] == run] for run in runs}
event_tables_by_run[runs[0]][0:1]

### Merger Rate Quantiles from O3 R&P Paper 

This section explains how we reproduce the 5%, 50%, and 95% quantiles of the BNS and NSBH merger rate, 

as reported in [O3 R&P, Table II](https://doi.org/10.1103/PhysRevX.13.011048), for our own simulation statistics.


## O3 R&P Table II (Last Column)

Reported BNS merger rates:

- **5% quantile:** 100 $\mathrm{Gpc}^{-3}\ \mathrm{yr}^{-1} $

- **50% quantile (median):** 240 $\mathrm{Gpc}^{-3}\ \mathrm{yr}^{-1} $

- **95% quantile:** 510 $\mathrm{Gpc}^{-3}\ \mathrm{yr}^{-1} $

In [ ]:
event_tables_by_run = {run: main_table[main_table["run"] == run] for run in runs}
event_tables_by_run[runs[0]][0:1]

In [ ]:
lo = 100
mid = 240
hi = 510

# 90% interval width (in log-normal)
(standard_90pct_interval,) = np.diff(stats.norm.interval(0.9))
log_target_rate_mu = np.log(mid)
log_target_rate_sigma = np.log(hi / lo) / standard_90pct_interval

print(log_target_rate_mu, log_target_rate_sigma)

### Compute effective rate for each run:

In [ ]:
log_simulation_effective_rate_by_run = {
    key: np.log(value.to_value(u.Gpc**-3 * u.yr**-1))
    for key, value in main_table.meta["effective_rate"].items()
}
log_simulation_effective_rate_by_run

### Compute $\mu$ for each run

 $\mu$ is calculated for each simulation run as:

$
\mu = \log(\text{target median}) + \log(\text{run duration}) - \log(\text{simulation effective rate}) + \log(\text{N or detected N})
$


Here we use both number of events and number detected (with known position).


In [ ]:
prob_quantiles = np.asarray([0.5, 0.05, 0.95])  # median, 5%, 95%
run_duration = 1.5  # years

mu = np.asarray(
    [
        log_target_rate_mu
        + np.log(run_duration)
        - log_simulation_effective_rate_by_run[run]
        + np.log(
            [
                np.sum(_)
                for _ in [
                    np.ones_like(event_tables_by_run[run]["objective_value"]),
                    event_tables_by_run[run]["detection_probability_known_position"],
                ]
            ]
        )
        for run in runs
    ]
)

mu

### Compute Poisson-Lognormal Rate Quantiles

This step calculates the quantile intervals for the merger rates for all runs.


In [ ]:
rate_quantiles = poisson_lognormal_rate_quantiles(
    prob_quantiles[np.newaxis, np.newaxis, :],
    mu.T[:, :, np.newaxis],
    log_target_rate_sigma,
)

idx = 0
print(f"Quantiles for first run {runs[idx]}:", rate_quantiles[idx])

In [ ]:
latex_rows = []
header = "Run & " + " & ".join(list(runs)) + r" \\"
latex_rows = [header]

for i, (label, row) in enumerate(
    zip(["Number of events selected", "Number of events detected"], rate_quantiles)
):
    formatted = [
        "${}_{{-{}}}^{{+{}}}$".format(
            int(np.rint(mid)), int(np.rint(mid - lo)), int(np.rint(hi - mid))
        )
        for mid, lo, hi in row
    ]
    line = " & ".join([label] + formatted) + r" \\"
    latex_rows.append(line)


latex_table = "\n".join(latex_rows)
print(latex_table)

In [ ]:
import numpy as np
from astropy import units as u
from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import z_at_value
from astropy.table import QTable


def format_row_latex(row):
    """Return a LaTeX table row for a given event."""

    # Format each field as required
    mass1 = np.format_float_positional(row["mass1"], 3, fractional=True)
    mass2 = np.format_float_positional(row["mass2"], 3, fractional=True)
    lon = np.format_float_positional(np.rad2deg(row["longitude"]), 4, fractional=True)
    lat = np.format_float_positional(
        np.rad2deg(row["latitude"]), 4, fractional=True, sign=True
    )
    distance = np.format_float_positional(row["distance"], 0, trim="-", fractional=True)
    area90 = np.format_float_positional(row["area(90)"], 0, trim="-", fractional=True)

    if row["objective_value"] >= 0.1:
        objective = "\\phantom{$<$}" + np.format_float_positional(
            row["objective_value"], 2, min_digits=2, fractional=True, trim="k"
        )
        detection = np.format_float_positional(
            row["detection_probability_known_position"],
            2,
            min_digits=2,
            fractional=True,
            trim="k",
        )
    else:
        objective = r"$<$0.10"
        detection = "---"
    # Construct the LaTeX table row
    return (
        f"{row['run']} & {row['coinc_event_id']} & {mass1} & {mass2} & "
        f"{lon} & {lat} & {distance} & {area90} & {objective} & {detection} \\\\"
    )


# --- Main processing ---
table = QTable.read("../data/events.ecsv")
zp1 = 1 + z_at_value(cosmo.luminosity_distance, table["distance"] * u.Mpc)
table["mass1"] /= zp1
table["mass2"] /= zp1
table = table[table["mass2"] <= 3]

n_show = 4  # Number of first/last events to show
rows = [format_row_latex(row) for row in table]

# Write full table to LaTeX file
with open("events.tex", "w") as f:
    for row in rows:
        print(row, file=f)

# For ReadTheDocs preview, print first and last 4 rows
print("% The first 4 events:")
for r in rows[:n_show]:
    print(r)
print("\n% The last 4 events:")
for r in rows[-n_show:]:
    print(r)